In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# !unzip '/content/drive/My Drive/LicensePlates/NEW/data/autoriaNumberplateOcrRu-2019-08-30.zip'

In [0]:
# !ls

In [0]:
# !cp -r '/content/drive/My Drive/LicensePlates/NEW/data/autoriaNumberplateOcrRu-2019-08-30' ocr_data

cp: cannot access '/content/drive/My Drive/LicensePlates/NEW/data/autoriaNumberplateOcrRu-2019-08-30/train/img': Input/output error
^C


In [0]:
# mcm
import os
from tensorflow.python.client import device_lib
import urllib.request
from tqdm import tqdm
import pathlib
latest_models = {
  "Detector": {
    "mrcnn": {
      "h5": {
        "gpu": "https://nomeroff.net.ua/models/mrcnn/mask_rcnn_numberplate_1000_2019_10_07.h5",
        "cpu": "https://nomeroff.net.ua/models/mrcnn/mask_rcnn_numberplate_1000_2019_10_07.h5"
      }
    }
  },
  "OptionsDetector": {
    "simple": {
      "h5": {
        "class_region": [
          "xx_unknown",
          "eu_ua_2015",
          "eu_ua_2004",
          "eu_ua_1995",
          "eu",
          "xx_transit",
          "ru",
          "kz",
          "eu-ua-fake-dnr",
          "eu-ua-fake-lnr",
          "ge"
        ],
        "gpu": "https://nomeroff.net.ua/models/options/0.3.x/numberplate_options_2019_10_07.h5",
        "cpu": "https://nomeroff.net.ua/models/options/0.3.x/numberplate_options_2019_10_07.h5"
      }
    }
  },
  "TextDetector": {
    "eu_ua_2004_2015": {
      "h5": {
        "gpu": "https://nomeroff.net.ua/models/ocr/ua/anpr_ocr_ua_17-gpu.h5",
        "cpu": "https://nomeroff.net.ua/models/ocr/ua/anpr_ocr_ua_17-cpu.h5"
      }
    },
    "eu_ua_1995": {
      "h5": {
        "gpu": "https://nomeroff.net.ua/models/ocr/ua-1995/anpr_ocr_ua-1995_2-gpu.h5",
        "cpu": "https://nomeroff.net.ua/models/ocr/ua-1995/anpr_ocr_ua-1995_2-cpu.h5"
      }
    },
    "eu": {
      "h5": {
        "cpu": "https://nomeroff.net.ua/models/ocr/eu/anpr_ocr_eu_2-cpu.h5",
        "gpu": "https://nomeroff.net.ua/models/ocr/eu/anpr_ocr_eu_2-gpu.h5"
      }
    },
    "ru": {
      "h5": {
        "cpu": "https://nomeroff.net.ua/models/ocr/ru/anpr_ocr_ru_6-cpu.h5",
        "gpu": "https://nomeroff.net.ua/models/ocr/ru/anpr_ocr_ru_6-gpu.h5"
      }
    },
    "kz": {
      "h5": {
        "cpu": "https://nomeroff.net.ua/models/ocr/kz/anpr_ocr_kz_4-cpu.h5",
        "gpu": "https://nomeroff.net.ua/models/ocr/kz/anpr_ocr_kz_4-gpu.h5"
      }
    },
    "ge": {
      "h5": {
        "cpu": "https://nomeroff.net.ua/models/ocr/ge/anpr_ocr_ge_3-cpu.h5",
        "gpu": "https://nomeroff.net.ua/models/ocr/ge/anpr_ocr_ge_3-gpu.h5"
      }
    }
  }
}
device_mode = 'gpu'
def download_latest_model(detector, model_name, ext="h5", mode = device_mode):
    if mode != "cpu" and mode != "gpu":
        mode = device_mode
    info = latest_models[detector][model_name][ext]
    info["path"] = os.path.join(os.path.dirname(os.path.realpath(__file__)), "./models", detector, model_name, os.path.basename(info[mode]))

    p = pathlib.Path(os.path.dirname(info["path"]))
    p.mkdir(parents=True, exist_ok=True)

    if not os.path.exists(info["path"]):
        #print("downloading model {} to {} ...".format(info[mode], info["path"]))
        download_url(info[mode], info['path'])

    return info

In [0]:
import imgaug as ia
from imgaug import augmenters as iaa
import numpy as np

ia.seed(1)

def aug(imgs):
    sometimes = lambda aug: iaa.Sometimes(0.5, aug)
    sometimes = lambda aug: iaa.Sometimes(0.5, aug)
    seq = iaa.Sequential(
        [
             sometimes(iaa.Crop(percent=(0, 0.01))),
             iaa.Affine(
                 scale={"x": (0.995, 1.01), "y": (0.995, 1.01)},
                 translate_percent={"x": (-0.01, 0.01), "y": (-0.01, 0.01)},
                 rotate=(-3, 3),
                 shear=(-3, 3),
                 #cval=(0, 255)
             ),
              iaa.SomeOf((0, 5),
              [
                      sometimes(iaa.OneOf([
                              iaa.OneOf([
                                  iaa.GaussianBlur((1, 1.2)),
                                  iaa.AverageBlur(k=(1, 3)),
                                  iaa.MedianBlur(k=(1, 3))
                              ]),
                              iaa.ContrastNormalization((0.5, 1.5), per_channel=0.5),
                              iaa.Grayscale(alpha=(0.0, 1.0)),
                              iaa.OneOf([
                                  iaa.EdgeDetect(alpha=(0, 0.7)),
                                  iaa.DirectedEdgeDetect(
                                      alpha=(0, 0.7), direction=(0.0, 1.0)
                                  ),
                              ]),
                      ])),
                      sometimes(iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5))),
                      sometimes(iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.005*255), per_channel=0.001)),
                      sometimes(iaa.Dropout((0.001, 0.01), per_channel=0.5)),
                      sometimes(iaa.Add((-10, 10), per_channel=0.5)),
                      sometimes(iaa.Multiply((0.5, 1.5), per_channel=0.5)),
                      sometimes(iaa.ElasticTransformation(alpha=(0.1, 0.2), sigma=0.05)),
                      sometimes(iaa.PiecewiseAffine(scale=(0.001, 0.005)))
                  ],
                  random_order=True
              )
         ],
        random_order=True
    )
    return seq.augment_images(imgs)

In [0]:
from os.path import join
import cv2
import os
import json
import numpy as np
from tensorflow.keras import backend as K
import random
import itertools


class TextImageGenerator:
    def __init__(self,
                 dirpath,
                 img_w, img_h,
                 batch_size,
                 downsample_factor,
                 letters,
                 max_text_len,
                 cname=""):

        self.CNAME = cname
        #print(self.CNAME)
        self.img_h = img_h
        self.img_w = img_w
        self.batch_size = batch_size
        self.max_text_len = max_text_len
        self.downsample_factor = downsample_factor
        self.letters = letters

        img_dirpath = join(dirpath, 'img')
        ann_dirpath = join(dirpath, 'ann')
        self.samples = []
        for filename in os.listdir(img_dirpath):
            name, ext = os.path.splitext(filename)
            if ext == '.png':
                img_filepath = join(img_dirpath, filename)
                json_filepath = join(ann_dirpath, name + '.json')
                description = json.load(open(json_filepath, 'r'))['description']
                if TextImageGenerator.is_valid_str(self, description):
                    self.samples.append([img_filepath, description])

        self.n = len(self.samples)
        self.indexes = list(range(self.n))
        self.cur_index = 0
        self.count_ep = 0
        self.letters_max = len(letters)+1

    def labels_to_text(self, labels):
        data = ''.join(list(map(lambda x: "" if x==self.letters_max else self.letters[int(x)], labels)))
        return data

    def text_to_labels(self, text):
        data = list(map(lambda x: self.letters.index(x), text))
        while len(data) < self.max_text_len:
            data.append(self.letters_max)
        return data

    def is_valid_str(self, s):
        for ch in s:
            if not ch in self.letters:
                return False
        return True

    def decode_batch(self, out):
        # Most change
        # For a real OCR application, this should be beam search with a dictionary
        # and language model.  For this example, best path is sufficient.
        ret = []
        for j in range(out.shape[0]):
            out_best = list(np.argmax(out[j, 2:], 1))
            out_best = [k for k, g in itertools.groupby(out_best)]
            outstr = ''
            for c in out_best:
                if c < len(self.letters):
                    outstr += self.letters[c]
            ret.append(outstr)
        return ret

    def build_data(self, aug_count=0):
        self.imgs = np.zeros((self.n, self.img_h, self.img_w))
        self.texts = []
        for i, (img_filepath, text) in enumerate(self.samples):
            img = cv2.imread(img_filepath)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            # CLAHE
            clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
            img = clahe.apply(img)

            img = cv2.resize(img, (self.img_w, self.img_h))
            img = img.astype(np.float32)
            img -= np.amin(img)
            img /= np.amax(img)
            # width and height are backwards from typical Keras convention
            # because width is the time dimension when it gets fed into the RNN
            self.imgs[i, :, :] = img
            self.texts.append(text)
        while aug_count:
            for i, (img_filepath, text) in enumerate(self.samples):
                img = cv2.imread(img_filepath)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                imgs = aug([img])
                img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
                img = imgs[0]
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

                # CLAHE
                clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
                img = clahe.apply(img)

                img = cv2.resize(img, (self.img_w, self.img_h))
                img = img.astype(np.float32)
                img -= np.amin(img)
                img /= np.amax(img)
                # width and height are backwards from typical Keras convention
                # because width is the time dimension when it gets fed into the RNN
                self.imgs[i, :, :] = img
                self.texts.append(text)
            aug_count -= 1


    def get_output_size(self):
        return len(self.letters) + 1

    def normalize(self, img):
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        img = cv2.resize(img, (self.IMG_W, self.IMG_H))

        # CLAHE
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        img = clahe.apply(img)

        img = img.astype(np.float32)
        img -= np.amin(img)
        img /= np.amax(img)
        img = [[[h] for h in w] for w in img.T]

        x = np.zeros((self.IMG_W, self.IMG_H, 1))
        x[:, :, :] = img
        return x

    def next_sample(self, is_random=1):
        self.cur_index += 1
        if self.cur_index >= self.n:
            self.count_ep += 1
            self.cur_index = 0
            if is_random:
                random.shuffle(self.indexes)
        return self.imgs[self.indexes[self.cur_index]], self.texts[self.indexes[self.cur_index]]

    def next_batch(self, is_random=1, input_name=None, output_name="ctc"):
        if not input_name:
            input_name = 'the_input_{}'.format(self.CNAME)
        while True:
            # width and height are backwards from typical Keras convention
            # because width is the time dimension when it gets fed into the RNN
            if K.image_data_format() == 'channels_first':
                X_data = np.ones([self.batch_size, 1, self.img_w, self.img_h])
            else:
                X_data = np.ones([self.batch_size, self.img_w, self.img_h, 1])

            Y_data = np.ones([self.batch_size, self.max_text_len])
            input_length = np.ones((self.batch_size, 1)) * (self.img_w // self.downsample_factor - 2)
            label_length = np.zeros((self.batch_size, 1))
            source_str = []

            for i in range(self.batch_size):
                img, text = self.next_sample(is_random)
                img = img.T
                if K.image_data_format() == 'channels_first':
                    img = np.expand_dims(img, 0)
                else:
                    img = np.expand_dims(img, -1)
                X_data[i] = img
                Y_data[i] = self.text_to_labels(text)
                source_str.append(text)
                label_length[i] = len(text)

            inputs = {
                '{}'.format(input_name): X_data,
                'the_labels_{}'.format(self.CNAME): Y_data,
                'input_length_{}'.format(self.CNAME): input_length,
                'label_length_{}'.format(self.CNAME): label_length,
                #'source_str': source_str
            }
            outputs = {'{}'.format(output_name): np.zeros([self.batch_size])}
            yield (inputs, outputs)

In [0]:
import tensorflow as tf
from tensorflow.python.framework import graph_io
from tensorflow.python.tools import freeze_graph
from tensorflow.core.protobuf import saver_pb2
from tensorflow.python.training import saver as saver_lib
from keras import backend as K

def convert_keras_to_freeze_pb(model, frozen_model_path):
        out_names = ",".join([layer.name.split(":")[0]  for layer in model.outputs])
        inp_names = ",".join([layer.name.split(":")[0]  for layer in model.inputs])
        print("OUTPUT: {}".format(out_names))
        print("INPUT: {}".format(inp_names))
        model.summary()
        K.set_learning_phase(0)
        sess = K.get_session()
        saver = saver_lib.Saver(write_version=saver_pb2.SaverDef.V2)
        checkpoint_path = saver.save(sess, '/content/drive/My Drive/LicensePlates/NEW/checkpoints/saved_ckpt', global_step=0, latest_filename='checkpoint_state')
        graph_io.write_graph(sess.graph, '.', '/content/drive/My Drive/LicensePlates/NEW/checkpoints/tmp.pb')
        freeze_graph.freeze_graph('/content/drive/My Drive/LicensePlates/NEW/checkpoints/tmp.pb', '',
                                False, checkpoint_path, out_names,
                                "/content/drive/My Drive/LicensePlates/NEW/checkpoints/save/restore_all",
                                "/content/drive/My Drive/LicensePlates/NEW/checkpoints/save/Const:0",
                                frozen_model_path,
                                False, "")

In [0]:
import tensorflow.keras
import tensorflow as tf

import os
from os.path import join
import json
import random
import numpy as np

from keras import backend as K
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Input, Dense, Activation
from keras.layers import Reshape, Lambda
from keras.layers.merge import add, concatenate
from keras.models import Model, load_model
from keras.optimizers import SGD
from keras.utils.data_utils import get_file
from keras.preprocessing import image
import keras.callbacks
from collections import Counter

from keras.layers import CuDNNGRU as GRUgpu
from keras.layers.recurrent import GRU as GRUcpu


import time

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

class OCR(TextImageGenerator):
    @classmethod
    def get_classname(cls):
        return cls.__name__

    def __init__(self):
        # Input parameters
        self.IMG_H = 64
        self.IMG_W = 128
        self.IMG_C = 1

        # Train parameters
        self.BATCH_SIZE = 32
        self.EPOCHS = 1

        # Network parameters
        self.CONV_FILTERS = 16
        self.KERNEL_SIZE = (3, 3)
        self.POOL_SIZE = 2
        self.TIME_DENSE_SIZE = 32
        self.RNN_SIZE = 512
        self.ACTIVATION = 'relu'
        self.DOWNSAMPLE_FACROT = self.POOL_SIZE * self.POOL_SIZE

        self.GRU = GRUcpu

        self.INPUT_NODE = "the_input_{}:0".format(type(self).__name__)
        self.OUTPUT_NODE = "softmax_{}/truediv:0".format(type(self).__name__)

    def get_counter(self, dirpath, verbose=1):
        dirname = os.path.basename(dirpath)
        ann_dirpath = join(dirpath, 'ann')
        letters = ''
        lens = []
        for filename in os.listdir(ann_dirpath):
            json_filepath = join(ann_dirpath, filename)
            description = json.load(open(json_filepath, 'r'))['description']
            lens.append(len(description))
            letters += description
        max_plate_length = max(Counter(lens).keys())
        if verbose:
            print('Max plate length in "%s":' % dirname, max_plate_length)
        return Counter(letters), max_plate_length

    def get_alphabet(self, train_path, test_path, val_path, verbose=1):
        c_val, max_plate_length_val     = self.get_counter(val_path)
        c_train, max_plate_length_train = self.get_counter(train_path)
        c_test, max_plate_length_test   = self.get_counter(test_path)

        letters_train = set(c_train.keys())
        letters_val = set(c_val.keys())
        letters_test = set(c_test.keys())
        if verbose:
            print("Letters train ", letters_train)
            print("Letters val ", letters_val)
            print("Letters test ", letters_test)

        if max_plate_length_val == max_plate_length_train:
            if verbose:
                print('Max plate length in train, test and val do match')
        else:
            raise Exception('Max plate length in train, test and val do not match')

        if letters_train == letters_val:
            if verbose:
                print('Letters in train, val and test do match')
        else:
            raise Exception('Letters in train, val and test do not match')

        self.letters = sorted(list(letters_train))
        self.max_text_len = max_plate_length_train
        if verbose:
            print('Letters:', ' '.join(self.letters))
        return self.letters, self.max_text_len

    def explainTextGenerator(self, train_dir, letters, max_plate_length, verbose=1):
        tiger = TextImageGenerator(train_dir, self.IMG_W, self.IMG_H, 1, self.POOL_SIZE * self.POOL_SIZE, letters, max_plate_length, cname=type(self).__name__)
        tiger.build_data()

        for inp, out in tiger.next_batch():
            print('Text generator output (data which will be fed into the neutral network):')
            print('1) the_input (image)')
            if K.image_data_format() == 'channels_first':
                img = inp['the_input_{}'.format(type(self).__name__)][0, 0, :, :]
            else:
                img = inp['the_input_{}'.format(type(self).__name__)][0, :, :, 0]

            #import matplotlib.pyplot as plt
            #plt.imshow(img.T, cmap='gray')
            #plt.show()
            print('2) the_labels (plate number): %s is encoded as %s' %
                  (tiger.labels_to_text(inp['the_labels_{}'.format(type(self).__name__)][0]), list(map(int, inp['the_labels_{}'.format(type(self).__name__)][0]))))
            print('3) input_length (width of image that is fed to the loss function): %d == %d / 4 - 2' %
                  (inp['input_length_{}'.format(type(self).__name__)][0], tiger.img_w))
            print('4) label_length (length of plate number): %d' % inp['label_length_{}'.format(type(self).__name__)][0])
            break

    def ctc_lambda_func(self, args):
        y_pred, labels, input_length, label_length = args
        # the 2 is critical here since the first couple outputs of the RNN
        # tend to be garbage:
        y_pred = y_pred[:, 2:, :]
        return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

    def save(self, path, verbose=1):
        if self.MODEL:
            self.MODEL.save(path)
            if verbose:
                print("SAVED TO {}".format(path))

    def test(self, verbose=1, random_state=1):
        if verbose:
            print("\nRUN TEST")
            start_time = time.time()
        net_inp = self.MODEL.get_layer(name='{}'.format(self.MODEL.layers[0].name)).input
        net_out = self.MODEL.get_layer(name='{}'.format(self.MODEL.layers[-1].name)).output

        err_c = 0
        succ_c = 0
        for inp_value, _ in self.tiger_test.next_batch(random_state, input_name=self.MODEL.layers[0].name, output_name=self.MODEL.layers[-1].name):
            bs = inp_value['the_input_{}'.format(type(self).__name__)].shape[0]
            X_data = inp_value['the_input_{}'.format(type(self).__name__)]
            net_out_value = self.SESS.run(net_out, feed_dict={net_inp:X_data})
            pred_texts = self.tiger_test.decode_batch(net_out_value)
            labels = inp_value['the_labels_{}'.format(type(self).__name__)]
            texts = []
            for label in labels:
                text = self.tiger_test.labels_to_text(label)
                texts.append(text)

            for i in range(bs):
                if (pred_texts[i] != texts[i]):
                    if verbose:
                        print('\nPredicted: \t\t %s\nTrue: \t\t\t %s' % (pred_texts[i], texts[i]))
                    err_c += 1
                else:
                    succ_c += 1
            break
        if verbose:
            print("Test processing time: {} seconds".format(time.time() - start_time))
        print("acc: {}".format(succ_c/(err_c+succ_c)))

    def predict(self, imgs, *argv):
        Xs = []
        for img in imgs:
            x = self.normalize(img)
            Xs.append(x)
        pred_texts = []
        if bool(Xs):
            net_out_value = self.MODEL.predict(np.array(Xs))
            #print(net_out_value)
            pred_texts = self.decode_batch(net_out_value)
        return pred_texts

    def load(self, path_to_model, mode="cpu", verbose = 0):
        if path_to_model =="latest":
            model_info = download_latest_model("TextDetector", self.get_classname(), mode=mode)
            path_to_model = model_info["path"]


        self.MODEL = load_model(path_to_model, compile=False)

        net_inp = self.MODEL.get_layer(name='{}'.format(self.MODEL.layers[0].name)).input
        net_out = self.MODEL.get_layer(name='{}'.format(self.MODEL.layers[-1].name)).output

        self.MODEL = Model(input=net_inp, output=net_out)

        if verbose:
            self.MODEL.summary()

        # the loss calc occurs elsewhere, so use a dummy lambda func for the loss
        #sgd = SGD(lr=0.02, decay=1e-6, momentum=0.9, nesterov=True, clipnorm=5)
        #self.MODEL.compile(loss={'{}'.format(model.layers[-1].name): lambda y_true, y_pred: y_pred}, optimizer=sgd)

        return self.MODEL

    def prepare(self, path_to_dataset, aug_count=0, verbose=1):
        self.SESS = tf.Session()
        K.set_session(self.SESS)

        train_path = os.path.join(path_to_dataset, "train")
        test_path  = os.path.join(path_to_dataset, "test")
        val_path   = os.path.join(path_to_dataset, "val")

        if verbose:
            print("GET ALPHABET")
        self.letters, max_plate_length = self.get_alphabet(train_path, test_path, val_path, verbose=verbose)

        if verbose:
            print("\nEXPLAIN DATA TRANSFORMATIONS")
            self.explainTextGenerator(train_path, self.letters, max_plate_length)

        if verbose:
            print("START BUILD DATA")
        self.tiger_train = TextImageGenerator(train_path, self.IMG_W, self.IMG_H, self.BATCH_SIZE, self.DOWNSAMPLE_FACROT, self.letters, max_plate_length, cname=type(self).__name__)
        self.tiger_train.build_data(aug_count=aug_count)
        self.tiger_val = TextImageGenerator(val_path,  self.IMG_W, self.IMG_H, self.BATCH_SIZE, self.DOWNSAMPLE_FACROT, self.letters, max_plate_length, cname=type(self).__name__)
        self.tiger_val.build_data()

        self.tiger_test = TextImageGenerator(test_path, self.IMG_W, self.IMG_H, len(os.listdir(os.path.join(test_path, "img"))), self.DOWNSAMPLE_FACROT, self.letters, max_plate_length, cname=type(self).__name__)
        self.tiger_test.build_data()
        if verbose:
            print("DATA PREPARED")

    def train(self, mode="gpu", is_random=1, model_path="/content/drive/My Drive/LicensePlates/NEW/models/second.h5", load=False, verbose=1):
        if mode == "gpu":
            self.GRU = GRUgpu
        if mode == "cpu":
            self.GRU = GRUcpu

        if verbose:
            print("\nSTART TRAINING")
        if K.image_data_format() == 'channels_first':
            input_shape = (1, self.IMG_W, self.IMG_H)
        else:
            input_shape = (self.IMG_W, self.IMG_H, 1)

        input_data = Input(name='the_input_{}'.format(type(self).__name__), shape=input_shape, dtype='float32')
        inner = Conv2D(self.CONV_FILTERS, self.KERNEL_SIZE, padding='same',
                       activation=self.ACTIVATION, kernel_initializer='he_normal',
                       name='conv1')(input_data)
        inner = MaxPooling2D(pool_size=(self.POOL_SIZE , self.POOL_SIZE ), name='max1')(inner)
        inner = Conv2D(self.CONV_FILTERS, self.KERNEL_SIZE, padding='same',
                       activation=self.ACTIVATION, kernel_initializer='he_normal',
                       name='conv2')(inner)
        inner = MaxPooling2D(pool_size=(self.POOL_SIZE , self.POOL_SIZE ), name='max2')(inner)

        conv_to_rnn_dims = (self.IMG_W // (self.POOL_SIZE  * self.POOL_SIZE), (self.IMG_H // (self.POOL_SIZE * self.POOL_SIZE)) * self.CONV_FILTERS)
        inner = Reshape(target_shape=conv_to_rnn_dims, name='reshape')(inner)

        # cuts down input size going into RNN:
        inner = Dense(self.TIME_DENSE_SIZE, activation=self.ACTIVATION, name='dense1')(inner)

        # Two layers of bidirecitonal GRUs
        # GRU seems to work as well, if not better than LSTM:
        gru_1 = self.GRU(self.RNN_SIZE, return_sequences=True, kernel_initializer='he_normal', name='gru1')(inner)
        gru_1b = self.GRU(self.RNN_SIZE, return_sequences=True, go_backwards=True, kernel_initializer='he_normal', name='gru1_b')(inner)
        gru1_merged = add([gru_1, gru_1b])
        gru_2 = self.GRU(self.RNN_SIZE, return_sequences=True, kernel_initializer='he_normal', name='gru2')(gru1_merged)
        gru_2b = self.GRU(self.RNN_SIZE, return_sequences=True, go_backwards=True, kernel_initializer='he_normal', name='gru2_b')(gru1_merged)

        # transforms RNN output to character activations:
        inner = Dense(self.tiger_train.get_output_size(), kernel_initializer='he_normal',
                      name='dense2')(concatenate([gru_2, gru_2b]))
        y_pred = Activation('softmax', name='softmax_{}'.format(type(self).__name__))(inner)
        Model(inputs=input_data, outputs=y_pred).summary()

        labels = Input(name='the_labels_{}'.format(type(self).__name__), shape=[self.tiger_train.max_text_len], dtype='float32')
        input_length = Input(name='input_length_{}'.format(type(self).__name__), shape=[1], dtype='int64')
        label_length = Input(name='label_length_{}'.format(type(self).__name__), shape=[1], dtype='int64')
        # Keras doesn't currently support loss funcs with extra parameters
        # so CTC loss is implemented in a lambda layer
        loss_out = Lambda(self.ctc_lambda_func, output_shape=(1,), name='ctc')([y_pred, labels, input_length, label_length])

        # clipnorm seems to speeds up convergence
        sgd = SGD(lr=0.02, decay=1e-6, momentum=0.9, nesterov=True, clipnorm=5)

        if load:
            model = load_model(model_path, compile=False)
        else:
            model = Model(inputs=[input_data, labels, input_length, label_length], outputs=loss_out)

        # the loss calc occurs elsewhere, so use a dummy lambda func for the loss
        model.compile(loss={'{}'.format(model.layers[-1].name): lambda y_true, y_pred: y_pred}, optimizer=sgd)

        # captures output of softmax so we can decode the output during visualization
        test_func = K.function([input_data], [y_pred])

        model.fit_generator(generator=self.tiger_train.next_batch(is_random, input_name=model.layers[0].name, output_name=model.layers[-1].name),
                            steps_per_epoch=self.tiger_train.n,
                            epochs=self.EPOCHS,
                            validation_data=self.tiger_val.next_batch(is_random, input_name=model.layers[0].name, output_name=model.layers[-1].name),
                            validation_steps=self.tiger_val.n)

        print("Models layers: ", [i.name for i in model.layers])
        # net_inp = model.get_layer(name='{}'.format(model.layers[0].name)).input
        # net_out = model.get_layer(name='{}'.format(model.layers[-1].name)).output
        self.MODEL = Model(inputs=input_data, outputs=y_pred)
        # self.MODEL = model
        return self.MODEL

    def load_frozen(self, FROZEN_MODEL_PATH, mode="cpu"):
        graph_def = tf.GraphDef()
        with tf.gfile.GFile(FROZEN_MODEL_PATH, "rb") as f:
            graph_def.ParseFromString(f.read())

        #print([x.name for x in graph_def.node])
        graph = tf.Graph()
        with graph.as_default():
            self.net_inp, self.net_out = tf.import_graph_def(
                graph_def, return_elements = [self.INPUT_NODE, self.OUTPUT_NODE]
            )

        sess_config = tf.ConfigProto()
        sess_config.gpu_options.allow_growth = True
        self.sess = tf.Session(graph=graph, config=sess_config)

    def frozen_predict(self, imgs):
        Xs = []
        for img in imgs:
            x = self.normalize(img)
            Xs.append(x)
        pred_texts = []
        if bool(Xs):
            net_out_value = self.sess.run([self.net_out], feed_dict={self.net_inp: np.array(Xs)})
            #print(net_out_value)
            pred_texts = self.decode_batch(net_out_value[0])
        return pred_texts

In [0]:
import sys
import warnings
warnings.filterwarnings('ignore')

import keras
keras.backend.clear_session()

# change this property
# NOMEROFF_NET_DIR = os.path.abspath('/content/drive/My Drive/LicensePlates/NEW/')
NOMEROFF_NET_DIR = os.path.abspath('/content/autoriaNumberplateOcrRu-2019-08-30/')

DATASET_NAME = "ru"
VERSION = "1"
MODE = "gpu"
# PATH_TO_DATASET = os.path.join(NOMEROFF_NET_DIR, "data/autoriaNumberplateOcrRu-2019-08-30/")
# PATH_TO_DATASET = os.path.join(NOMEROFF_NET_DIR, "data_short/ocr-2020/")
PATH_TO_DATASET = os.path.join(NOMEROFF_NET_DIR)
RESULT_MODEL_PATH = os.path.join(NOMEROFF_NET_DIR, "models/", 'anpr_ocr_{}_{}-{}.h5'.format(DATASET_NAME, VERSION, MODE))

FROZEN_MODEL_PATH = os.path.join(NOMEROFF_NET_DIR, "models/", 'anpr_ocr_{}_{}-{}.pb'.format(DATASET_NAME, VERSION, MODE))

sys.path.append(NOMEROFF_NET_DIR)



In [0]:
class ru_train(OCR):
    def __init__(self):
        OCR.__init__(self)
        # only for usage model
        # in train generate automaticly
        # self.letters = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "A", "B", "C", "E", "H", "K", "M", "O", "P", "T", "X", "Y"]
        
        self.EPOCHS = 3

In [0]:
ocrTextDetector = ru_train()
model = ocrTextDetector.prepare(PATH_TO_DATASET, aug_count=0)

GET ALPHABET
Max plate length in "val": 9
Max plate length in "train": 9
Max plate length in "test": 9
Letters train  {'Y', 'H', '2', '4', '5', 'K', '6', '9', 'O', 'M', '1', 'P', '7', '8', '0', 'A', 'E', 'X', 'C', '3', 'B', 'T'}
Letters val  {'Y', 'H', '2', '4', '5', 'K', '6', '9', 'O', 'M', '1', 'P', '7', '8', '0', 'A', 'X', 'E', 'C', '3', 'B', 'T'}
Letters test  {'Y', 'H', '2', '4', '5', 'K', '6', '9', 'O', 'M', '1', 'P', '7', '8', '0', 'A', 'X', 'E', 'C', '3', 'B', 'T'}
Max plate length in train, test and val do match
Letters in train, val and test do match
Letters: 0 1 2 3 4 5 6 7 8 9 A B C E H K M O P T X Y

EXPLAIN DATA TRANSFORMATIONS
Text generator output (data which will be fed into the neutral network):
1) the_input (image)
2) the_labels (plate number): T763PP35 is encoded as [19, 7, 6, 3, 18, 18, 3, 5, 23]
3) input_length (width of image that is fed to the loss function): 30 == 128 / 4 - 2
4) label_length (length of plate number): 8
START BUILD DATA
DATA PREPARED


In [0]:
model = ocrTextDetector.train(mode=MODE, is_random=1)


START TRAINING
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input_ru_train (InputLayer) (None, 128, 64, 1)   0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 64, 16)  160         the_input_ru_train[0][0]         
__________________________________________________________________________________________________
max1 (MaxPooling2D)             (None, 64, 32, 16)   0           conv1[0][0]                      
__________________________________________________________________________________________________
conv2 (Conv2D)                  (None, 64, 32, 16)   2320        max1[0][0]                       
____________________________________________________________________________

In [0]:
RESULT_MODEL_PATH = '/content/drive/My Drive/LicensePlates/NEW/models/96percent.h5'
ocrTextDetector.save(RESULT_MODEL_PATH, verbose=True)


In [0]:
ocrTextDetector.test(verbose=True)


RUN TEST

Predicted: 		 K707KP580
True: 			 K707KP58

Predicted: 		 K733KB78
True: 			 X733KB78

Predicted: 		 T135OA3
True: 			 T135OA33

Predicted: 		 M056HX29
True: 			 H056HX29

Predicted: 		 A302TT16525
True: 			 A302TT152

Predicted: 		 H120HH32
True: 			 H120HH39

Predicted: 		 X518XX652
True: 			 X518XX61

Predicted: 		 M882MM11
True: 			 M882MM12

Predicted: 		 B639MY114
True: 			 B639MY116

Predicted: 		 H008YH64
True: 			 H008HH64

Predicted: 		 T639AP197
True: 			 T689AP197

Predicted: 		 T522TT33
True: 			 T522TT39

Predicted: 		 X193BH196
True: 			 X193BH197

Predicted: 		 E2B2PO46
True: 			 E282PO46

Predicted: 		 K770KY335
True: 			 K770KY33

Predicted: 		 E555BY192
True: 			 E555BY102

Predicted: 		 H209CY228
True: 			 H209CY22

Predicted: 		 B648AB29
True: 			 H648AB29

Predicted: 		 Y013YY74
True: 			 Y013YY26
Test processing time: 0.5713357925415039 seconds
acc: 0.9613034623217923


In [0]:
# Upload model
RESULT_MODEL_PATH = '/content/drive/My Drive/LicensePlates/NEW/models/96percent.h5'
model =ocrTextDetector.load(RESULT_MODEL_PATH)


In [0]:
import numpy as np
import cv2
def hand_image_prep(filename:str):
    img = cv2.imread(filename, 0)
    print(img.shape)
    # img = img[np.newaxis, ...]
    # img = cv2.cvtColor(img, cv2.COLOR_GBR2GRAY)

    # CLAHE
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    img = clahe.apply(img)

    img = cv2.resize(img, (64, 128))
    img = img.astype(np.float32)
    img -= np.amin(img)
    img /= np.amax(img)
    # width and height are backwards from typical Keras convention
    # because width is the time dimension when it gets fed into the RNN
    # img[1, :, :] = img
    # img = img[np.newaxis, ...]
    print(img.shape)

    return img
img = hand_image_prep('/content/drive/My Drive/LicensePlates/plates_examples/car_plate_rus_5_cut.jpg')
a = ocrTextDetector.predict([img])


In [0]:
img = cv2.imread('/content/drive/My Drive/LicensePlates/plates_examples/car_plate_rus_4_cut.jpg')

a = ocrTextDetector.predict([img])

In [0]:
a

['P719XK77']

In [0]:
# Converting h5(keras) to pb(tf)
import keras
keras.backend.clear_session()
model = ocrTextDetector.load(RESULT_MODEL_PATH)
# model = ocrTextDetector.load('/content/drive/My Drive/LicensePlates/NEW/models/ru_my.h5')
convert_keras_to_freeze_pb(model, '/content/drive/My Drive/LicensePlates/NEW/models/96_percent.pb')

OUTPUT: softmax_ru_train/truediv
INPUT: the_input_ru_train
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input_ru_train (InputLayer) (None, 128, 64, 1)   0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 64, 16)  160         the_input_ru_train[0][0]         
__________________________________________________________________________________________________
max1 (MaxPooling2D)             (None, 64, 32, 16)   0           conv1[0][0]                      
__________________________________________________________________________________________________
conv2 (Conv2D)                  (None, 64, 32, 16)   2320        max1[0][0]                       
_________________________________

In [0]:
from math import exp 
exp(16.11)

9919370.3057168